In [1]:
import torch
import numpy as np
import pandas as pd
import os
import datetime
from datetime import date, timedelta
from graph import Graph
from utils.data_utils import data_to_numpy

In [2]:
unlabelled = pd.read_csv('data/la_train_grid_with_weather.csv')
training_data = pd.read_csv('data/LA_DATA_2018_02_to_2018_06.csv')


In [5]:
def format_unlabeled_dataset(df):
    df = df.rename({'day' : 'DATE'}, axis = 1)

    cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew',
                 'precipitation_duration' , 'mean_aod','min_aod','max_aod', 'visibility',                                               
                 'Latitudes','Longitudes','DATE']

    df = df[cols]
    return df
    

In [6]:
unlabelled = format_unlabeled_dataset(unlabelled)

In [7]:
# node_cols = 'visibility', 'precipitation_depth']
node_cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew', 'precipitation_duration' , 'mean_aod','min_aod','max_aod', 'pm25']                                                 
edge_cols = ['wind_x', 'wind_y']
date_range = sorted(training_data['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

def assign_id(weather_data):
    ids = np.zeros(len(weather_data['Latitudes']))
    node_pos = list(weather_data.groupby(['Latitudes', 'Longitudes']).groups)
    for i, node in enumerate(node_pos):
        indices_lats = weather_data['Latitudes'].values == node_pos[i][0]
        indices_longs = weather_data['Longitudes'].values == node_pos[i][1]
        indices = indices_lats & indices_longs
        ids[np.argwhere(indices)] = int(i+1)
    weather_data['STATION'] = ids
    return weather_data

def get_unlabelled_subset(unlabeled_df, dates, node_cols, edge_cols, meta_cols):
    cols = list(set(node_cols + edge_cols + meta_cols))
    unlabeled_df = unlabeled_df[cols]
    unlabeled_df = unlabeled_df[unlabeled_df['DATE'].isin(dates)]
    unlabeled_df = assign_id(unlabeled_df)
    unlabeled_df['pm25'] = [np.nan for _ in range(len(unlabeled_df))]
    return unlabeled_df






# graph_node_features, graph_edge_features, graph_labels

In [8]:
node_cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew', 'precipitation_duration' , 'mean_aod','min_aod','max_aod']                                                 
edge_cols = ['wind_x', 'wind_y']
meta_cols = ['DATE', 'Latitudes', 'Longitudes']
date_range = sorted(training_data['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

unlabeled_df = get_unlabelled_subset(unlabelled, date_range,  node_cols, edge_cols, meta_cols)

In [9]:
dataset = unlabeled_df.append(training_data, ignore_index=True)

In [12]:

date_range = sorted(dataset['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
#rethink : CAN I PUT IT DIRECTELY INTO A DGN GRAPH